In [1]:
# import copy
# import gym
import torch
import random
# import functools
from stable_baselines3.common.logger import configure

import numpy as np
import pandas as pd
import itertools
import sys

sys.path.append("/home/miw039/scalable_rl_portfolio_management/FinRL-dev")
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.env_portfolio_optimization.env_portfolio_optimization import PortfolioOptimizationEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from finrl import config_tickers
from finrl.config import INDICATORS
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split


device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(device)

ModuleNotFoundError: No module named 'finrl'

In [ ]:
raw_df = YahooDownloader(start_date = '2009-01-01',
                        end_date = '2020-07-01',
                        ticker_list = ['ticker']).fetch_data()

In [ ]:
def preprocess(df):
    fe = FeatureEngineer(
        use_technical_indicator=True,
        tech_indicator_list = INDICATORS,
        use_vix=False,
        use_turbulence=True,
        user_defined_feature = False)

    processed = fe.preprocess_data(df)

    list_ticker = processed["tic"].unique().tolist()
    list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
    combination = list(itertools.product(list_date,list_ticker))

    processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
    processed_full = processed_full[processed_full['date'].isin(processed['date'])]
    processed_full = processed_full.sort_values(['date','tic'])
    processed_full = processed_full.fillna(0)
    return processed_full

raw_df = YahooDownloader(start_date = '2009-01-01',
                        end_date = '2020-07-01',
                        ticker_list = config_tickers.DOW_30_TICKER).fetch_data()

raw_df.to_csv('data/raw_data.csv', index=False)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

Shape of DataFrame:  (84221, 8)


In [ ]:
raw_df = raw_df[["date", "tic", "close", "high", "low"]]
print('PROCESSING DATA')
df = preprocess(raw_df)
print(df.shape)
features = list(df.columns)
features.remove('date')
features.remove('tic')
# df.iloc[:, :100].to_csv('processed_data.csv', index=False)

In [3]:
from stable_baselines3.common.callbacks import BaseCallback

class ActionLoggerCallback(BaseCallback):
    def __init__(self, verbose=0):
        super(ActionLoggerCallback, self).__init__()

    def _on_step(self) -> bool:
        if self.num_timesteps % 10 == 0:
            rewards = self.locals["reward"]
            
            print(f"Step: {self.num_timesteps}")
            print(f"Rewards: {rewards}")
            print("------")
        
        return True

In [5]:
from stable_baselines3 import SAC


# set up logger
# now = datetime.now().strftime('%Y%m%d-%Hh%M')
# algo = 'sac'
# tmp_path = f'log/{algo}/0'
# new_logger = configure(tmp_path, ["stdout", "csv", "tensorboard"])

environment = PortfolioOptimizationEnv(
    df,
    initial_amount=100000,
    comission_fee_pct=0.0025,
    time_window=50,
    reward_scaling=1e-4,
    features=["close", "high", "low", "close_30_sma"])

model = SAC(
        policy='MlpPolicy',
        env=environment,
        tensorboard_log='log/sac/0',
        device=device,
        buffer_size=2000,
)

# Train the model
print('BEGIN TRAINING')
model.learn(total_timesteps=1000, progress_bar=True)

Normalizing ['close', 'high', 'low', 'close_30_sma'] by previous time...

/dsmlp/home-fs03/95/495/miw039/.conda/envs/ai4finance/lib/python3.8/site-packages/stable_baselines3/common/vec_env/
patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to 
Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, 
which could potentially cause issues.
  warnings.warn(

BEGIN TRAINING

LiveError: Only one live display may be active at once

In [3]:
TOP_BRL = [
    "VALE3.SA", "PETR4.SA", "ITUB4.SA", "BBDC4.SA",
    "BBAS3.SA", "RENT3.SA", "LREN3.SA", "PRIO3.SA",
    "WEGE3.SA", "ABEV3.SA"
]

portfolio_raw_df = YahooDownloader(start_date = '2011-01-01',
                                end_date = '2019-12-31',
                                ticker_list = TOP_BRL).fetch_data()
portfolio_raw_df

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Shape of DataFrame:  (22330, 8)


Price,date,close,high,low,open,volume,tic,day
0,2011-01-03,5.156441,8.728203,8.630313,8.632311,576145,ABEV3.SA,0
1,2011-01-03,6.230808,15.900000,15.690000,15.750000,6626800,BBAS3.SA,0
2,2011-01-03,5.983674,11.927362,11.724237,11.809763,10862336,BBDC4.SA,0
3,2011-01-03,12.436995,16.591017,16.330231,16.392323,11016129,ITUB4.SA,0
4,2011-01-03,6.178610,7.845370,7.656995,7.656995,4017796,LREN3.SA,0
...,...,...,...,...,...,...,...,...
22325,2019-12-30,12.024283,30.709999,30.150000,30.549999,22111600,PETR4.SA,0
22326,2019-12-30,6.601397,6.832000,6.570000,6.780000,8933500,PRIO3.SA,0
22327,2019-12-30,41.995987,48.063652,47.078293,47.735199,2714322,RENT3.SA,0
22328,2019-12-30,33.202404,53.860001,53.200001,53.650002,11928100,VALE3.SA,0


In [4]:
df_portfolio = portfolio_raw_df[["date", "tic", "close", "high", "low"]]
df_portfolio

Price,date,tic,close,high,low
0,2011-01-03,ABEV3.SA,5.156441,8.728203,8.630313
1,2011-01-03,BBAS3.SA,6.230808,15.900000,15.690000
2,2011-01-03,BBDC4.SA,5.983674,11.927362,11.724237
3,2011-01-03,ITUB4.SA,12.436995,16.591017,16.330231
4,2011-01-03,LREN3.SA,6.178610,7.845370,7.656995
...,...,...,...,...,...
22325,2019-12-30,PETR4.SA,12.024283,30.709999,30.150000
22326,2019-12-30,PRIO3.SA,6.601397,6.832000,6.570000
22327,2019-12-30,RENT3.SA,41.995987,48.063652,47.078293
22328,2019-12-30,VALE3.SA,33.202404,53.860001,53.200001


In [13]:
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
import pandas as pd
import itertools
from finrl.config import INDICATORS

def preprocess(df):
    fe = FeatureEngineer(
        use_technical_indicator=True,
        tech_indicator_list = INDICATORS,
        use_vix=True,
        use_turbulence=True,
        user_defined_feature = False)

    processed = fe.preprocess_data(df)

    list_ticker = processed["tic"].unique().tolist()
    list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
    combination = list(itertools.product(list_date,list_ticker))

    processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
    processed_full = processed_full[processed_full['date'].isin(processed['date'])]
    processed_full = processed_full.sort_values(['date','tic'])
    processed_full = processed_full.fillna(0)
    return processed_full

df_portfolio_processed = preprocess(df_portfolio)

[*********************100%***********************]  1 of 1 completed

Successfully added technical indicators
Shape of DataFrame:  (2262, 8)
Successfully added vix


Successfully added turbulence index


In [15]:
features = list(df_portfolio_processed.columns)
features.remove('date')
features.remove('tic')

environment = PortfolioOptimizationEnv(
        df_portfolio_processed,
        initial_amount=100000,
        comission_fee_pct=0.0025,
        time_window=50,
        features=features
    )


Normalizing ['close', 'high', 'low', 'macd', 'boll_ub', 'boll_lb', 'rsi_30', 'cci_30', 'dx_30', 'close_30_sma', 'close_60_sma', 'vix', 'turbulence'] by previous time...


In [12]:
from stable_baselines3.common.logger import configure

algo = 'sac'
sb_env, _ = environment.get_sb_env()
agent = DRLAgent(env=sb_env)
model = agent.get_model(algo)

# set up logger
tmp_path = f'log/{algo}'
new_logger = configure(tmp_path, ["stdout", "csv", "tensorboard"])
# Set new logger
model.set_logger(new_logger)

# Train the model
trained_model = agent.train_model(model=model, tb_log_name=algo, total_timesteps=10000)

/dsmlp/home-fs03/95/495/miw039/.conda/envs/ai4finance/lib/python3.8/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


{'batch_size': 64, 'buffer_size': 100000, 'learning_rate': 0.0001, 'learning_starts': 100, 'ent_coef': 'auto_0.1'}
Using cpu device
Logging to log/sac
Initial portfolio value:100000
Final portfolio value: 80894.5
Final accumulative portfolio value: 0.808945
Maximum DrawDown: -0.7088562915444212
Sharpe ratio: -0.0011301809308906865


findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.


Initial portfolio value:100000
Final portfolio value: 71648.53125
Final accumulative portfolio value: 0.7164853125
Maximum DrawDown: -0.7437782331292782
Sharpe ratio: -0.061245402400810375


findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.


Initial portfolio value:100000
Final portfolio value: 74479.984375
Final accumulative portfolio value: 0.74479984375
Maximum DrawDown: -0.7634529451805727
Sharpe ratio: -0.0430456574458893


findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.


Initial portfolio value:100000
Final portfolio value: 81773.375
Final accumulative portfolio value: 0.81773375
Maximum DrawDown: -0.7375919478713387
Sharpe ratio: 0.007750266723072075


findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.


-------------------------------------
| time/              |              |
|    episodes        | 4            |
|    fps             | 16           |
|    time_elapsed    | 520          |
|    total_timesteps | 8736         |
| train/             |              |
|    actor_loss      | -17.1        |
|    critic_loss     | 0.0162       |
|    ent_coef        | 0.0422       |
|    ent_coef_loss   | -58.7        |
|    learning_rate   | 0.0001       |
|    n_updates       | 8635         |
|    reward          | -0.010254327 |
-------------------------------------
